<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/The_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Collection**

In [ ]:
#https://towardsdatascience.com/a-labelling-tool-to-easily-extract-and-label-wikipedia-data-63f58e2e76ae
#https://medium.com/algoanalytics/automatic-labelling-of-text-for-nlp-5270e70a2f5f